In [ ]:
import os
import re
import sys
import time
import datetime
import psycopg2
import numpy as np
import pandas as pd

from scipy import stats
from matplotlib import pyplot as plt

In [ ]:
%matplotlib 
%load_ext autoreload
%autoreload 2

In [ ]:
user = 'keith'
host = 'localhost'
dbname = 'cypy2'
conn = psycopg2.connect(user=user, host=host, dbname=dbname)

### Analyzing timepoints

In [ ]:
d = pd.read_sql(
    'select heart_rate, power, cadence, speed from proc_records r '
    'left join metadata m on m.activity_id = r.activity_id '
    'where activity_type = \'ride\'',
    conn)
d = d.dropna(axis=0, how='any')

In [ ]:
d.shape

In [ ]:
data = []
for ind, column in enumerate(d.columns):
    vals = np.concatenate(tuple(d[column].values))
    data.append(vals)
data = np.array(data)

In [ ]:
mask = np.any(pd.isna(data), axis=0)
data = data[:, ~mask].astype(float)

In [ ]:
# for heart rate
hr = data[0, :]
_ = plt.hist(vals[~np.isnan(vals)], bins=np.arange(60, 190, 2))

In [ ]:
# for power
vals = data[2, :]
_ = plt.hist(vals[~np.isnan(vals) & (vals < 500) & (vals > 0)], bins=np.arange(0, 500, 2))

In [ ]:
hr = data[0, :]
power = data[1, :]
hr = hr[power > 0]
power = power[power > 0]

_ = plt.hist2d(hr, power, bins=[np.arange(80, 190, 2), np.arange(50, 400, 10)])

In [ ]:
cadence = data[2, :]
speed = data[3, :]
speed = speed[cadence > 0]
cadence = cadence[cadence > 0]

h = plt.hist2d(cadence, speed, bins=[np.arange(50, 90, 1), np.arange(1, 10, .03)])

In [ ]:
plt.imshow(np.log(h[0] + 1), aspect='auto')

In [ ]:
plt.scatter(cadence, speed)